<a href="https://colab.research.google.com/github/Soothysay/Big_Data_Class/blob/main/PySpark%2BHIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the Required Files

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.0.3-bin-hadoop2.7'

# Launching Spark Session

In [ ]:
from pyspark.sql import SparkSession
# Launching Spark Session with Hive
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .enableHiveSupport()\
        .getOrCreate()

In [ ]:
spark

# Optional

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-07-08 15:16:35--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.228.108.156, 44.193.52.174, 3.223.240.69, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.228.108.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.16’

ngrok-stable-linux- 100%[===================>]  13.19M  31.4MB/s    in 0.4s    

2021-07-08 15:16:36 (31.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.16’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   
{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://bbdc2ebb25e7.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":25,"gauge":0,"rate1":8.798996708838241e-26,"rate5":

# Creating Spark DataFrame

In [ ]:
df = spark.read.csv('/content/exchg_rate.csv',header=True,inferSchema=True)

In [ ]:
df.show()

+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC|      CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|
+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+
|       SKK|     BDT| Slovakian Krona|     Bangladesh Taka|                  B|            excg rate|              01/11/05|             30/11/05|  3.182271075|
|       SKK|     USD| Slovakian Krona|     American Dollar|                  B|            excg rate|              01/11/05|             30/11/05|  0.044820719|
|       SKK|   *UNK*| Slovakian Krona|                null|                  B|            excg rate|              01/11/05|             30/11/05|          0.0|
|       SKK|     IDR| Slovakian Kr

In [ ]:
df.printSchema()

root
 |-- FROMCRNCCD: string (nullable = true)
 |-- TOCRNCCD: string (nullable = true)
 |-- CRNCFROMLONGDESC: string (nullable = true)
 |-- CRNCTOLONGDESC: string (nullable = true)
 |-- CRNCEXCHGRATETYPECD: string (nullable = true)
 |-- CRNCEXCHGRATETYPEDESC: string (nullable = true)
 |-- CRNCEXCHGRATEEFFSTRTDT: string (nullable = true)
 |-- CRNCEXCHGRATEEFFENDDT: string (nullable = true)
 |-- CRNCEXCHGRATE: double (nullable = true)



In [ ]:
df.count()

725671

In [ ]:
# Summary of Data
df.describe().show()

+-------+----------+--------+-----------------+-----------------+-------------------+---------------------+----------------------+---------------------+------------------+
|summary|FROMCRNCCD|TOCRNCCD| CRNCFROMLONGDESC|   CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|     CRNCEXCHGRATE|
+-------+----------+--------+-----------------+-----------------+-------------------+---------------------+----------------------+---------------------+------------------+
|  count|    725671|  725671|           714598|           714598|             725671|               725671|                725671|               725671|            725671|
|   mean|      null|    null|             null|             null|               null|                 null|                  null|                 null| 8617.573998703845|
| stddev|      null|    null|             null|             null|               null|                 null|                  null|          

# Some Basic Spark DataFrame Operations

In [ ]:
set1=df.select('FROMCRNCCD','TOCRNCCD','CRNCEXCHGRATE')

In [ ]:
set1.show()

+----------+--------+-------------+
|FROMCRNCCD|TOCRNCCD|CRNCEXCHGRATE|
+----------+--------+-------------+
|       SKK|     BDT|  3.182271075|
|       SKK|     USD|  0.044820719|
|       SKK|   *UNK*|          0.0|
|       SKK|     IDR|  403.3864743|
|       SKK|     MYR|   0.13894423|
|       SKK|     PYG|   210.657381|
|       SKK|     HRK|  0.236909325|
|       SKK|     UYU|  0.896414387|
|       SKK|     KRW|  51.54382727|
|       SKK|     ZWD|   0.16449204|
|       SKK|     CZK|  0.800369861|
|       SKK|     THB|  1.344621581|
|       SKK|     AUD|  0.047179705|
|       SKK|     VND|  941.2351066|
|       SKK|     HKD|  0.349601611|
|       SKK|     SGD|  0.056025899|
|       SKK|     CAD|  0.044820719|
|       SKK|     GBP|   0.02721256|
|       SKK|     ZAR|  0.336155395|
|       SKK|     TRL|  71713.15098|
+----------+--------+-------------+
only showing top 20 rows



In [ ]:
df.filter(df.FROMCRNCCD=='INR').show()

+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC|      CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|
+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+
|       INR|     BDT|    Indian Rupee|     Bangladesh Taka|                  B|            excg rate|              01/11/05|             30/11/05|  1.577777778|
|       INR|     USD|    Indian Rupee|     American Dollar|                  B|            excg rate|              01/11/05|             30/11/05|  0.022222222|
|       INR|   *UNK*|    Indian Rupee|                null|                  B|            excg rate|              01/11/05|             30/11/05|          0.0|
|       INR|     IDR|    Indian Ru

# Create Temporary View (HIVE Table)

In [ ]:
# Converting Spark Dataframe to Hive Table
df.write.mode("overwrite").saveAsTable("test_table2")

In [ ]:
#Instances where INR Present
INR=spark.sql("select * from test_table2  a where a.FROMCRNCCD=='INR'")

In [ ]:
INR.show()

+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC|      CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|
+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+
|       INR|     BDT|    Indian Rupee|     Bangladesh Taka|                  B|            excg rate|              01/11/05|             30/11/05|  1.577777778|
|       INR|     USD|    Indian Rupee|     American Dollar|                  B|            excg rate|              01/11/05|             30/11/05|  0.022222222|
|       INR|   *UNK*|    Indian Rupee|                null|                  B|            excg rate|              01/11/05|             30/11/05|          0.0|
|       INR|     IDR|    Indian Ru

In [ ]:
#Instances where INR to Euro Present
INR_EUR=spark.sql("select * from test_table2  a where a.FROMCRNCCD=='INR' AND a.TOCRNCCD=='EUR'")

In [ ]:
INR_EUR.show()

+----------+--------+----------------+-------------------+-------------------+---------------------+----------------------+---------------------+-------------+-----------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC|     CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|             BACK|
+----------+--------+----------------+-------------------+-------------------+---------------------+----------------------+---------------------+-------------+-----------------+
|       INR|     EUR|    Indian Rupee|EMU currency (Euro)|                  B|            excg rate|              01/11/05|             30/11/05|  0.015873016|     62.999999496|
|       INR|     EUR|    Indian Rupee|EMU currency (Euro)|                  B|            excg rate|              01/12/05|             31/12/05|  0.015873016|     62.999999496|
|       INR|     EUR|    Indian Rupee|EMU currency (Euro)|                  B|            excg rate|          

In [ ]:
#Instances where INR to USD Present
INR_USD=spark.sql("select * from test_table2  a where a.FROMCRNCCD=='INR' AND a.TOCRNCCD=='USD'")

In [ ]:
INR_USD.show()

+----------+--------+----------------+---------------+-------------------+---------------------+----------------------+---------------------+-------------+------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC| CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|        BACK|
+----------+--------+----------------+---------------+-------------------+---------------------+----------------------+---------------------+-------------+------------+
|       INR|     USD|    Indian Rupee|American Dollar|                  B|            excg rate|              01/11/05|             30/11/05|  0.022222222| 45.00000045|
|       INR|     USD|    Indian Rupee|American Dollar|                  B|            excg rate|              01/12/05|             31/12/05|  0.022222222| 45.00000045|
|       INR|     USD|    Indian Rupee|American Dollar|                  B|            excg rate|              01/01/06|             31/01/06|  0.022222222|

# Transformation Function and creating a new column in DataFrames

In [ ]:
# Converting INR to USD
INR_USD=INR_USD.withColumn("BACK", 1/INR_USD.CRNCEXCHGRATE)

In [ ]:
INR_USD.show()

+----------+--------+----------------+---------------+-------------------+---------------------+----------------------+---------------------+-------------+------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC| CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|        BACK|
+----------+--------+----------------+---------------+-------------------+---------------------+----------------------+---------------------+-------------+------------+
|       INR|     USD|    Indian Rupee|American Dollar|                  B|            excg rate|              01/11/05|             30/11/05|  0.022222222| 45.00000045|
|       INR|     USD|    Indian Rupee|American Dollar|                  B|            excg rate|              01/12/05|             31/12/05|  0.022222222| 45.00000045|
|       INR|     USD|    Indian Rupee|American Dollar|                  B|            excg rate|              01/01/06|             31/01/06|  0.022222222|

In [ ]:
# Converting INR to Euro
INR_EUR=INR_EUR.withColumn("BACK", 1/INR_USD.CRNCEXCHGRATE)

In [ ]:
INR_EUR.show()

+----------+--------+----------------+-------------------+-------------------+---------------------+----------------------+---------------------+-------------+-----------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC|     CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|             BACK|
+----------+--------+----------------+-------------------+-------------------+---------------------+----------------------+---------------------+-------------+-----------------+
|       INR|     EUR|    Indian Rupee|EMU currency (Euro)|                  B|            excg rate|              01/11/05|             30/11/05|  0.015873016|     62.999999496|
|       INR|     EUR|    Indian Rupee|EMU currency (Euro)|                  B|            excg rate|              01/12/05|             31/12/05|  0.015873016|     62.999999496|
|       INR|     EUR|    Indian Rupee|EMU currency (Euro)|                  B|            excg rate|          

In [ ]:
# If we are doing this to whole DataFrame
df=df.withColumn("BACK", 1/df.CRNCEXCHGRATE)

In [ ]:
df.show()

+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+--------------------+
|FROMCRNCCD|TOCRNCCD|CRNCFROMLONGDESC|      CRNCTOLONGDESC|CRNCEXCHGRATETYPECD|CRNCEXCHGRATETYPEDESC|CRNCEXCHGRATEEFFSTRTDT|CRNCEXCHGRATEEFFENDDT|CRNCEXCHGRATE|                BACK|
+----------+--------+----------------+--------------------+-------------------+---------------------+----------------------+---------------------+-------------+--------------------+
|       SKK|     BDT| Slovakian Krona|     Bangladesh Taka|                  B|            excg rate|              01/11/05|             30/11/05|  3.182271075|  0.3142409858971552|
|       SKK|     USD| Slovakian Krona|     American Dollar|                  B|            excg rate|              01/11/05|             30/11/05|  0.044820719|  22.311110180985718|
|       SKK|   *UNK*| Slovakian Krona|                null|                  B|           

In [69]:
from pyspark.sql.functions import col, avg
# Average of Euro Currency
INR_EUR.agg(avg(col("BACK"))).show()
# Average of USD Currency
INR_USD.agg(avg(col("BACK"))).show()

+-----------------+
|        avg(BACK)|
+-----------------+
|79.23437279809256|
+-----------------+

+----------------+
|       avg(BACK)|
+----------------+
|66.7437176038998|
+----------------+

